# Step by step guide for converting tensorflow weight files to tflite model
## Make sure to run on GPU
## Make sure to uncomment and install all the dependencies

In [ ]:
%env PYTHONPATH = # /env/python

env: PYTHONPATH=# /env/python


In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh


--2023-10-04 04:35:51--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76120962 (73M) [application/x-sh]
Saving to: ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’

Miniconda3-py38_4.1 100%[===================>]  72.59M   225MB/s    in 0.3s    

2023-10-04 04:35:51 (225 MB/s) - ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’ saved [76120962/76120962]



In [ ]:
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh

In [ ]:
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
!conda update conda

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [ ]:
!conda create -n myenv python=3.6

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install tensorflow==1.15
pip install Pillow

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 /content/convert.py   # convert.py is in folder directory, This will generate the model.pb file from .weights file, change class names file accordingly, you can pass class.txt with all the class names



# This part is for converting the model.pb file to tflite

In [67]:
import tensorflow.compat.v1 as tf
gf = tf.GraphDef()
m_file = open('/content/frozen_darknet_yolov3_model.pb','rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt', 'a') as the_file:
     for n in gf.node:
         the_file.write(n.name+'\n')

file = open('somefile.txt','r')
data = file.readlines()
output = data[len(data)-1]
print("Output array = ", output)

file.seek ( 0 )
input=file.readline()
print("Input array = ", input)

Output array =  output_boxes

Input array =  inputs



In [70]:
import tempfile
import os
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/frozen_darknet_yolov3_model.pb',
    input_arrays = ['inputs'],   # Here, 'inputs' is the value of input array from Step 7b
    output_arrays = ['output_boxes'], # Here, 'output_boxes' is the value of output array from Step 7b
    input_shapes={'inputs': [1, 416, 416, 3]} # Here, 'inputs' is the value of input array from Step 7b
)


converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert to TFLite Model
tflite_model = converter.convert()

# _, dynamic_tflite_path = tempfile.mkstemp('.tflite')
tflite_path='/content/model.tflite'
tflite_model_size = open(tflite_path, 'wb').write(tflite_model)
tf_model_size = os.path.getsize('/content/frozen_darknet_yolov3_model.pb')
print('TensorFlow Model is  {} bytes'.format(tf_model_size))
print('TFLite Model is      {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

TensorFlow Model is  246517284 bytes
TFLite Model is      62138672 bytes
Post training dynamic range quantization saves 184378612 bytes


In [69]:
!ls -lh {dynamic_tflite_path}

-rw------- 1 root root 60M Oct  4 09:45 /tmp/tmp8q6nqofj.tflite


In [ ]:
import glob

tflite_file = ''
tflite_file = tflite_file.join(glob.glob("/tmp/*.tflite")) # Find the temp file of type .tflite
tflite_file = tflite_file[5:len(tflite_file)]  # Extract .tflite file name.

%cd ..
%cd ../tmp
!ls
# # os.rename(tflite_file, 'detect.tflite') #Rename the temp .tflite to detect.tflite
# %mv tmpry_03hf4.tflite ../content/gdrive/MyDrive/YOLOv3_TFLite/  # Move detect.tflite to Goolge Drive

In [77]:
# !pip install opencv-python
# !pip install tensorflow
!pip install Pillow

  Obtaining dependency information for Pillow from https://files.pythonhosted.org/packages/cd/6d/07566c00ddb116a0eca1a623abda12da81099a6ff3200e5e6b7e2d3c8c2b/Pillow-10.0.1-cp38-cp38-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 28.0 MB/s eta 0:00:00


In [86]:
!python /test_tflite.py  # this is for inferencing with generated tflite file


2023-10-04 10:18:43.021946: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
[{'name': 'output_boxes', 'index': 407, 'shape': array([    1, 10647,     8], dtype=int32), 'shape_signature': array([    1, 10647,     8], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Predicted image saved as output.jpg


In [2]:
!pip install tool


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for tool: filename=tool-0.8.0-py3-none-any.whl size=70547 sha256=fe14d77a3ccf2fb592cd5c852b339aefd80baaf974e4467f6d7159db9f812dd5
  Stored in directory: /root/.cache/pip/wheels/77/e3/85/55d80f5c19223845ba8c7e581b152b047394e9487416ba853b
Successfully built tool


In [19]:
!git clone https://github.com/linghu8812/tensorrt_inference.git # this repo is for converting the model to onnx format

Cloning into 'tensorrt_inference'...
remote: Enumerating objects: 1443, done.
remote: Counting objects: 100% (1439/1439), done.
remote: Compressing objects: 100% (694/694), done.
remote: Total 1443 (delta 783), reused 1318 (delta 685), pack-reused 4
Receiving objects: 100% (1443/1443), 12.01 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (783/783), done.


In [1]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 96.2 MB/s eta 0:00:00


In [ ]:

!python3 export_onnx.py --cfg_file /content/gesture.cfg --weights_file /content/drive/MyDrive/yc_gesture_final.weights --output_file yolov3.onnx --strides 32 16 8 --neck FPN
# The above line converts the weights and cfg from trained tensorflow model to onnx format

In [3]:
!git clone https://github.com/linghu8812/YOLOv3-TensorRT.git

Cloning into 'YOLOv3-TensorRT'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 52
Receiving objects: 100% (55/55), 1.49 MiB | 2.06 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [4]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=615df6d840167ec7981366f48568d9a161f3684d74b221a05f36c46f11596652
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda


In [13]:
!pip install nvidia-tensorrt==7.2.* --index-url https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement nvidia-tensorrt==7.2.* (from versions: 8.4.0.6, 8.4.1.5, 8.4.2.4, 8.4.3.1)
ERROR: No matching distribution found for nvidia-tensorrt==7.2.*


In [12]:
 !python3 /content/YOLOv3-TensorRT/onnx_to_tensorrt.py --onnx_file /content/yolov3.onnx --engine_file yolov3.trt


Namespace(width=608, height=608, batch_size=1, dataset='coco_labels.txt', int8=False, calib_file='yolo_calibration.cache', onnx_file='/content/yolov3.onnx', engine_file='yolov3.trt', image_file='/content/try.png', result_file='gesture_bboxes.png')
[10/05/2023-07:26:51] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/05/2023-07:26:51] [TRT] [W] The implicit batch dimension mode has been deprecated. Please create the network with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag whenever possible.
Traceback (most recent call last):
  File "/content/YOLOv3-TensorRT/onnx_to_tensorrt.py", line 179, in <module>
    main(args.width, args.height, args.batch_size, args.dataset, args.int8, args.calib_file, args.onnx_file, args.engine_file,
  File "/content/YOLOv3-TensorRT/onn